In [10]:
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [26]:
from torch.utils.data import DataLoader
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator
from sentence_transformers import InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
import tqdm
import pickle

In [2]:
import pytrec_eval
pytrec_eval.supported_measures


{'11pt_avg',
 'G',
 'P',
 'Rndcg',
 'Rprec',
 'Rprec_mult',
 'binG',
 'bpref',
 'gm_bpref',
 'gm_map',
 'infAP',
 'iprec_at_recall',
 'map',
 'map_cut',
 'ndcg',
 'ndcg_cut',
 'ndcg_rel',
 'num_nonrel_judged_ret',
 'num_q',
 'num_rel',
 'num_rel_ret',
 'num_ret',
 'recall',
 'recip_rank',
 'relative_P',
 'relstring',
 'runid',
 'set_F',
 'set_P',
 'set_map',
 'set_recall',
 'set_relative_P',
 'success',
 'utility'}

In [40]:
from itertools import count
from collections import defaultdict
from sentence_transformers import  LoggingHandler, SentenceTransformer, evaluation, util, models
factory = lambda c=count(): 0
pdict = defaultdict(factory)
qdict = defaultdict(factory)
qRels = defaultdict(list)
data_folder =os.path.join(os.path.expanduser('~'), '.cache/MSMARCO')
qrel_filepath = os.path.join(data_folder, 'qrels.train.tsv')
with open(qrel_filepath) as fIn:
    for line in fIn:
        qid, _, pid, _ = line.strip().split('\t')
        pdict[pid]+=1
        qdict[qid]+=1
        qRels[int(qid)].append(int(pid))
        
data_folder =os.path.join(os.path.expanduser('~'), '.cache/MSMARCO')
qrel_filepath = os.path.join(data_folder, 'queries.train.tsv')
queriesText=defaultdict(str)
with open(qrel_filepath) as fIn:
    for line in fIn:
        qid, queries = line.strip().split('\t')
        queriesText[qid]=queries
model_name="msmarco-distilbert-base-tas-b"
# model_name="output/mse-huggingfaceHard10EpochDist/171600"
# model_name="../output/log/0"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
model = SentenceTransformer(model_name)
qEmb=model.encode(list(queriesText.values()),batch_size=32,show_progress_bar=True)

: 

In [32]:
# model_name = args.model_name



In [1]:
import BatchExpLaunch.results_org as rog
import pandas as pd
import os
# scriptPath=os.path.dirname(os.path.abspath(__file__))
# os.chdir(scriptPath+"/..")
path_root="../output/mse-huggingfaceHard10EpochDist"
path_root="../output/mse-huggingfaceHard10EpochCo-con"
path_root="../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37"
# path_root="../../output/train_bi-encoder-mnrl-distilbert-base-uncased-margin_3.0-2022-11-30_11-40-45"
# OutputPath=os.path.join(path_root,"result")
_,MeanResult=rog.get_result_df(path_root,rerun=True)
Result,MeanResult=rog.get_result_df(path_root,groupby=None)
df = pd.concat(Result, axis=0)
df["sum"]=df["DL19ndcg_cut_10"]+df["DL20ndcg_cut_10"]
# print(df)
print(df.loc[df["sum"].idxmax()])
print(df[df["sum"]>1.4])


../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/20000/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/100000/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/190000/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/150000/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/10000/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/180000/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/235740/Eval/AggResults.jjson
../../output/train_bi-encoder-margin_mse-distilbert-base-uncased-batch_size_64-2022-11-30_11-40-37/90000/E

<Figure size 640x480 with 0 Axes>

In [6]:
run={"1":{"1":43,"2":23,"3":120}}

In [13]:
import csv
from eval import runsDict2trec,runsDict2Msmarco
outfile="../LogslurmRun/rnk.list"

runsDict2Msmarco(run, outfile)  

Generating ranklists: 100%|██████████| 1/1 [00:00<00:00, 11008.67it/s]


In [31]:
d="fdsa\tfdsa\tgfa\tgad"
d.strip().split()

['fdsa', 'fdsa', 'gfa', 'gad']

In [32]:
from str2bool import str2bool


In [33]:
str2bool("True")

True

In [2]:

import sys
import json
import os
os.chdir("../")
from collections import defaultdict
from utils.data import loadMSCorpus,loadDevMSqueries,loadDevRelMSQrels
# scriptPath=os.path.dirname(os.path.abspath(__file__))

from eval import *
from typing import List, Tuple, Dict, Set, Callable
from sentence_transformers import  LoggingHandler, SentenceTransformer, evaluation, util, models
import numpy as np
class evaluatorSbert(evaluation.InformationRetrievalEvaluator):
    def __init__(self,*args, **kwargs):
        super().__init__(corpus={1:"None"},*args, **kwargs)

    def compute_metrics(self, queries_result_dict):
        # Init score computation values
        num_hits_at_k = {k: 0 for k in self.accuracy_at_k}
        precisions_at_k = {k: [] for k in self.precision_recall_at_k}
        recall_at_k = {k: [] for k in self.precision_recall_at_k}
        MRR = {k: 0 for k in self.mrr_at_k}
        ndcg = {k: [] for k in self.ndcg_at_k}
        AveP_at_k = {k: [] for k in self.map_at_k}

        # Compute scores on results
        counter=0
        for query_itr in list(queries_result_dict.keys()):
            counter+=1
            query_id = query_itr

            # Sort scores
            top_hits = sorted(queries_result_dict[query_itr], key=lambda x: x['score'], reverse=True)
            query_relevant_docs = self.relevant_docs[query_id]

            # Accuracy@k - We count the result correct, if at least one relevant doc is accross the top-k documents
            for k_val in self.accuracy_at_k:
                for hit in top_hits[0:k_val]:
                    if hit['corpus_id'] in query_relevant_docs:
                        num_hits_at_k[k_val] += 1
                        break

            # Precision and Recall@k
            for k_val in self.precision_recall_at_k:
                num_correct = 0
                for hit in top_hits[0:k_val]:
                    if hit['corpus_id'] in query_relevant_docs:
                        num_correct += 1

                precisions_at_k[k_val].append(num_correct / k_val)
                recall_at_k[k_val].append(num_correct / len(query_relevant_docs))

            # MRR@k
            for k_val in self.mrr_at_k:
                for rank, hit in enumerate(top_hits[0:k_val]):
                    if hit['corpus_id'] in query_relevant_docs:
                        MRR[k_val] += 1.0 / (rank + 1)
                        break

            # NDCG@k
            for k_val in self.ndcg_at_k:
                predicted_relevance = [1 if top_hit['corpus_id'] in query_relevant_docs else 0 for top_hit in top_hits[0:k_val]]
                true_relevances = [1] * len(query_relevant_docs)

                ndcg_value = self.compute_dcg_at_k(predicted_relevance, k_val) / self.compute_dcg_at_k(true_relevances, k_val)
                ndcg[k_val].append(ndcg_value)

            # MAP@k
            for k_val in self.map_at_k:
                num_correct = 0
                sum_precisions = 0

                for rank, hit in enumerate(top_hits[0:k_val]):
                    if hit['corpus_id'] in query_relevant_docs:
                        num_correct += 1
                        sum_precisions += num_correct / (rank + 1)

                avg_precision = sum_precisions / min(k_val, len(query_relevant_docs))
                AveP_at_k[k_val].append(avg_precision)

        # Compute averages
        for k in num_hits_at_k:
            num_hits_at_k[k] /= len(self.queries)

        for k in precisions_at_k:
            precisions_at_k[k] = np.mean(precisions_at_k[k])

        for k in recall_at_k:
            recall_at_k[k] = np.mean(recall_at_k[k])

        for k in ndcg:
            ndcg[k] = np.mean(ndcg[k])

        for k in MRR:
            MRR[k] /= len(self.queries)

        for k in AveP_at_k:
            AveP_at_k[k] = np.mean(AveP_at_k[k])

        print(counter)
        return {'accuracy@k': num_hits_at_k, 'precision@k': precisions_at_k, 'recall@k': recall_at_k, 'ndcg@k': ndcg, 'mrr@k': MRR, 'map@k': AveP_at_k}


trecRun= "/home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/training_ms-marco_cross-encoder-v2-microsoft-MiniLM-L12-H384-uncased-2022-12-03_13-25-05/985000/Eval/MSDev/output.trec.csv"
trecRun = "/home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/350000/Eval/MSDev/output.trec.csv"

# class sbertEval()

def fromTrecRuntoSbertRun(trecRun):
    sbertRun=defaultdict(list)
    for line in open(trecRun):
            query_id, _, doc_id, rank, score, _ = line.strip().split(' ')
            sbertRun[query_id].append({'corpus_id': doc_id, 'score': float(score)})
    return sbertRun

sbertRun=fromTrecRuntoSbertRun(trecRun)
data_folder =os.path.join(os.path.expanduser('~'), '.cache/MSMARCO')

dev_queries=loadDevMSqueries(data_folder)
dev_rel_docs=loadDevRelMSQrels(data_folder,dev_queries)
# Read passages
# corpus=loadMSCorpus(data_folder)
ir_evaluator = evaluatorSbert(queries=dev_queries,  relevant_docs=dev_rel_docs,
                                                        show_progress_bar=True,
                                                        precision_recall_at_k=[10, 100],
                                                        name="msmarco dev")
# ir_evaluator=LoadMSDevEvaluator(data_folder)
ir_evaluator.compute_metrics(sbertRun)

2022-12-05 20:25:31 - Read queries: queries.dev.tsv
2022-12-05 20:25:31 - Dev query size 6980
2022-12-05 20:25:31 - Load qrels.dev.tsv
6980


{'accuracy@k': {1: 0.2340974212034384,
  3: 0.4232091690544413,
  5: 0.5131805157593123,
  10: 0.6151862464183381},
 'precision@k': {10: 0.06305157593123208, 100: 0.008312320916905444},
 'recall@k': {10: 0.6041189111747851, 100: 0.788538681948424},
 'ndcg@k': {10: 0.40842490750190563},
 'mrr@k': {10: 0.3506348069313686},
 'map@k': {100: 0.3532380094198648}}

In [5]:
qq=list(dev_rel_docs.keys())
len(qq)

6980

In [ ]:
./utils/trec_eval-9.0.7/trec_eval  -M 10 -m  recip_rank -m ndcg /home/collab/u1368791/.cache/MSMARCO/qrels.dev.tsv  /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/350000/Eval/MSDev/output.trec.csv

In [ ]:
./utils/trec_eval-9.0.7/trec_eval  -m  recip_rank -m ndcg_cut.10 /home/collab/u1368791/.cache/MSMARCO/qrels.dev.tsv  /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/350000/Eval/MSDev/output.trec.csv

In [ ]:
./utils/trec_eval-9.0.7/trec_eval  -M 10 -m  recip_rank -m ndcg /home/collab/u1368791/.cache/MSMARCO/qrels.dev.tsv  /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/350000/Eval/MSDev/output.trec.csv

In [ ]:
./utils/trec_eval-9.0.7/trec_eval  -M 10 -m  recip_rank -m ndcg /home/collab/u1368791/.cache/MSMARCO_DL19Pass/2019qrels-pass.txt /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/650000/Eval/DL19/output.ms.csv

In [ ]:
./utils/trec_eval-9.0.7/trec_eval  -m  recip_rank -m ndcg_cut.10 /home/collab/u1368791/.cache/MSMARCO_DL19Pass/2019qrels-pass.txt /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/650000/Eval/DL19/output.trec.csv

In [ ]:
python  ./utils/msmarco_passage_eval.py  /home/collab/u1368791/.cache/MSMARCO_DL19Pass/2019qrels-pass.txt /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/650000/Eval/DL19/output.ms.csv

In [ ]:
python  ./utils/evalSbert.py /home/collab/u1368791/.cache/MSMARCO_DL19Pass/2019qrels-pass.txt /home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-margin_mse--home-collab-u1368791-largefiles-TaoFiles-sentence-transformers-examples-training-ms_marco-output-mrr0.36Biencoder-235740--batch_size_64-2022-12-03_23-48-15/650000/Eval/DL19/output.trec.csv

In [1]:
tt=getattr("print")

TypeError: getattr expected at least 2 arguments, got 1

In [3]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'tt=getattr("print")', 'tt("fds")', 'globals()'],
 '_oh': {},
 '_dh': ['/home/collab/u1368791/largefiles/TaoFiles/sentence-transformers/examples/training/ms_marco/NeuralIR/drafts'],
 'In': ['', 'tt=getattr("print")', 'tt("fds")', 'globals()'],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fd2a9571f10>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x7fd2a9598310>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x7fd2a9598310>,
 '_': '',
 '__': '',
 '___': '',
 'os': <module 'os' from '/home/collab/u1368791/anaconda3/envs/sbert/lib/python3.7/os.py'>,
 'sys': <module 'sys' (built-in)>,
 '__vsc_ipynb_file__': '/home/collab/u136

In [2]:
tt("fds")

NameError: name 'tt' is not defined